# Find Unique
 - Deliverables:
     - findUnique.py - 50 points ( We will test this code)
     - Notebook with Inspection materials and Results ( 5 pts )
     - required input from STDIN
     - required output to STDOUT
     - output must be sorted by tRNA header

## Sets and tRNA

Mitochondrial tRNA are encoded in the mitochondrial genome and account for ~10% of the coding space, yet over 50% of mitochondrial genomic disease have their origin in this molecule. These molecules are transcribed, processed, modified, and ultimately folded to produce these mature adapters between the mitochondrial transcription and translation processes. Defects in mt.tRNA maturation reflect additional disease states .. if only we could identify those mutations. We are working on such a device, though we first need to use it to identify abundance of these molecules in a mixed population. Are there any unique subsequences among the 22 mt.tRNA that can be used as "tags"? If so, we can count those tags to assess abundance.

## Assignment

Write a python command line program that reads a file of fasta sequences from STDIN, finds the unique subsequences that occur in each single tRNA such that no members of this set occur among any of the other tRNA sets. Each of those 22 sets should be minimized, such that no member of that unique subsequence set is a substring of any other member of this set. [ Unique and Essential]

As an example, let's say that both ACG and AAACGA are in a unique set. Since ACG is a substring of AAACGA we would remove AAACGA. [ ACG is Essential ]

Use Python sets for this assignment[__required__]. Not only will your code be smaller, but it will be more likely to work. The union, intersection and difference operators will be quite useful.

## Rough design plan...
0) Consider a tRNA as an object. In thi scase we would then have 22 objects that would need to be able to communicate with each other.

1) compute the set of all substrings from each tRNA sequence. [powerset] I will refer to a set of substrings as a tRNAset.

2) for each tRNAset, compute the union of all other tRNA sets, and then remove that union from the current tRNAset. Notice that this union operation finds all of the substrings from all other tRNA. If any of those are present in your current tRNA, then they are not unique ! [ Unique]

3) for each unique tRNAset, it now contains the truly unique ones, along with any extensions of that subsequence. If, for example, it was found that G only occurred in a single tRNA, then adding an A onto that G must also be unique because it has a G in it. We only want the minimal form.. G. [ Essential]

4) Remove spaces from the header line before sorting and printing. This will make your output a little prettier.

5) make sure to remove any alignment characters from your initial sequences. These characters are periods(.), underscores(\_), or dashes(-) . You will find many new characters in the sequence other than {ACGU} - leave these in place, they are modified bases.

## Report
Print a report that contains items as follows. 

 - Line 1: the tRNA name
 - Line 2: the tRNA sequence ( with the alignment characters removed)
 - lines 3-80 or so, each unique element.

These unique elements need to be ordered by their starting position in the tRNA sequence, and should be spaced over so they are directly under that subsequence in the original tRNA. This looks like an alignment, but you can find where it belongs by using the string.find() method. Include dots in all positions to the left, serving as alignment guides for your reader. [ see sample output below ]

Do this for all of the 22 tRNA sequences.

Print the tRNA out as above, sorted by the header line.  

## Hints:

__use sets and the set operators - this is required !__

your final code will be under 100 lines.

Do most of your coding using methods defined in a class and then instantiated as objects. 

The sequences include characters that are just alignment characters. They are not part of the sequence and must be removed. [-\_\.] are alignment characters. 

When removing items from a tRNAset, don't do this while iterating through that set. Also, when building a unique set you will need the original contents of all other tRNAsets. So.. build a new set to keep the unique contents, or keep track of the elements you intend to delete later. Notice that you build the union of all other tRNA, and this happens 22 times - these unions are all distinct from each other. Example, consider 4 sets, A,B,C,D.  we would compute the set B, C, D to use against set A, and we would compute the set A, C, D to use against B. A and B need to not change while we are doing this computation.

There are cases where a unique element is present multiple times in a single tRNA. This sequence is unique to this tRNA and should be reported in its multiple locations ( same sequence on multiple lines of the output).

This is a command line program, though it does not have any optional parameters. You really don't need the commandline class. Your input comes from STDIN and output goes to STDOUT. Use the FastaReader for input and print statement for output. If you are going to use jupyter to develop your code, you can add a filename to fastareader for your testing.

## Submission

Submit code using Canvas. As always, work with your group and write your own code.

In [1]:
#!/usr/bin/env python3
# Name: Akash Pandit (aspandit)
# Group: None


"""
    Input for STDIN (in order to compile):
    python3 findUnique.py <bos-tRNA-7.fa >bos-tRNA-7-output.txt
"""


from sequenceAnalysis import FastAreader

class findUnique:
    def __init__(self):
        """Reads tRNA from the inputted Fasta file. Adds the power set of each sequence to a list"""
        self.pSets, self.uniques, self.headerDict = [], [], {}

    def genPSets(self, sequence: str) -> None:
        """Appends all unique tRNA substrings (a powerset)"""
        powerSets = set()
        for index in range(len(sequence)):
            size = len(sequence)
            while size > index:
                powerSets.add(sequence[index:size])
                size -= 1

        self.pSets.append(powerSets)

    def findUniques(self):
        """
        Looks for duplicate tRNA sequences and removes from all but the appriopiate set to
        obtain a unique set of tRNA subsequences.
        """
        for pSet in self.pSets:
            union = set()
            copyList = self.pSets.copy()
            copySet = pSet.copy()  # Make duplicate of the possible power set
            copyList.remove(copySet)  # Remove power set from list.

            for powerSet in copyList: # removes the duplicate elements between union and copySet
                union = union.union(powerSet)  # The union of all the other tRNA sets.

            copySet.difference_update(union)  # Update the copySet, removing elements found in union.

            newSet = copySet.copy()
            for s1 in copySet:  # checks prior strings to identify more possible substrings
                uniqueSet = copySet.copy()
                uniqueSet.remove(s1)
                for s2 in uniqueSet:
                    if s1 in s2:
                        if len(s1) < len(s2):
                            newSet.discard(s2)  # Gets rid of the larger substrings.

            self.uniques.append(newSet)

    def print(self) -> None:
        """Prints to STDOUT (CL in this case)"""
        for index in range(0,len(self.headerDict)):
            header, sequence = self.headerDict[index]
            print(header)
            print(sequence)
            for i in range(len(sequence)): 
                for substr in self.uniques[index]:
                    if substr == sequence[len(substr)]:
                        print('.'*i + substr)

def main():
    """Runs above code to find and print header, sequence and substrings from the fasta file"""

    # init objects & counter var
    tRNA = findUnique()
    fastaFile = FastAreader()
    count = 0

    # generate psets & increment counter
    for header, sequence in fastaFile.readFasta():
        filteredSequence = sequence.replace('-', '').replace('_', '')  # makes sure only valid characters are being used
        tRNA.headerDict[count] = [header, filteredSequence]
        tRNA.genPSets(filteredSequence)  # generate & append pset to self.psets
        count += 1

    # find & print unique seqs
    tRNA.findUniques()
    tRNA.print()


if __name__ == "__main__":
    main()

# Sample Output

In [ ]:
tRNA|Lys|∃UU|Bostaurus|mitochondrial
CACUAAGA"LCUAUAUAGCACPAACCU∃UU6AGUUAGAGAUUGAGAGCCAU"UACUCUCCUUGGUGACCA
CACU
.ACUA
...UAA
....AAG
.......A"
.........L
..........CUAU
............AUAU
..............AUAG
...............UAGC
.................GCA
.....................P
......................AAC
.......................ACCU
...........................∃
..............................6
...............................AGU
................................GUU
.................................UUA
..................................UAGA
...................................AGAGA
......................................GAU
.......................................AUU
........................................UUGA
.........................................UGAG
..........................................GAGAG
............................................GAGC
..............................................GCC
................................................CAU
..................................................U"
..................................................."U
....................................................UAC
.....................................................ACUC
.......................................................UCU
.........................................................UCC
...........................................................CUU
..............................................................GG
...............................................................GUG
.................................................................GAC
..................................................................ACCA